In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv("../input/facial-keypoints-detection/training.zip")

In [ ]:
test=pd.read_csv("../input/facial-keypoints-detection/test.zip")

In [ ]:
train.head

In [ ]:
train.columns

In [ ]:
x_train=train['Image']
y_train=train.drop('Image',axis=1)

In [ ]:
import numpy as np

In [ ]:
x_train_array=[]
for i in x_train:
    a=i.split(" ")
    a= list(map(int, a)) 
    a=np.asarray(a).reshape(96,96)
    x_train_array.append(a)
    
    
    

we need to remove null values and fill na values and need to convert x_train from string to the type we need

In [ ]:
x_train_=np.asarray(x_train_array)
x_train_final=np.expand_dims(x_train_,axis=3)
x_train_final.shape

x_train_final is the array of images we need now lets work on y_train

In [ ]:
y_train

In [ ]:
for i in y_train.columns:
    y_train[str(i)]=y_train[str(i)].fillna(y_train[str(i)].mean(axis=0))

In [ ]:
y_train.isna().sum()

next 2 blocks just to visualize

In [ ]:
x=[]
y=[]
count=2
for i in y_train.iloc[0]:
    if(count%2==0):
        x.append(i)
    else:
        y.append(i)
    count+=1
    

In [ ]:
plt.imshow(np.squeeze(x_train_final[0],axis=2))
plt.scatter(x,y,color='red')

now i split into trainng and validation sets

In [ ]:
x_t,x_v,y_t,y_v=train_test_split(x_train_final,y_train,test_size=0.2,shuffle=True)

In [ ]:
y_t

        now we create out model

In [ ]:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense,Conv2D,MaxPool2D,Flatten,Dropout
from keras import backend as K

In [ ]:
def func():
    input_layer=Input(shape=(96,96,1))
   
    x1=Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',name='x1',kernel_initializer=tf.keras.initializers.GlorotNormal())(input_layer)
    x2=Conv2D(64,kernel_size=(3,3),activation='relu',padding='same',name='x2',kernel_initializer=tf.keras.initializers.GlorotNormal())(x1)
    x3=Conv2D(64,kernel_size=(3,3),activation='relu',padding='same',name='x3',kernel_initializer=tf.keras.initializers.GlorotNormal())(x2)
    x4=x2+x3
    x5=Conv2D(128,kernel_size=(3,3),activation='relu',padding='same',name='x5',kernel_initializer=tf.keras.initializers.GlorotNormal())(x4)
    x6=Conv2D(128,kernel_size=(3,3),activation='relu',padding='same',name='x6',kernel_initializer=tf.keras.initializers.GlorotNormal())(x5)
    x7=x5+x6
    
    return input_layer,x7
    

In [ ]:
def func2():
    
    il,x8=func()
    
    x9=MaxPool2D((2,2),name='x9')(x8)
    x10=Dropout(0.2,name='x10')(x9)
    x11=Conv2D(256,kernel_size=(3,3),padding='same',name='x11',activation='relu')(Conv2D(256,kernel_size=(3,3),padding='same',activation='relu')(x10))
    x12=MaxPool2D((2,2),name='x12')(x11)
    x13=Dropout(0.2,name='x13')(x12)
    x14=Conv2D(256,kernel_size=(3,3),padding='same',name='x14',activation='relu')(Conv2D(512,kernel_size=(3,3),padding='same',activation='relu')(x13))
   
    x17=Flatten(name='x17')(x14)
    
    x18=Dense(100,name='x18')(x17)
    x19=Dense(30,name='x19')(x18)
    return Model(inputs=il,outputs=x19)
    
    

In [ ]:
model=func2()

In [ ]:
model.summary()

In [ ]:
class callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs={}):
        if(logs.get('accuracy')>0.95):
            print('95% accuracy')
            self.model.stop_training=True
            
        

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy','mae'],loss=tf.keras.losses.MeanSquaredError())

In [ ]:
call=callbacks()

In [ ]:
earlystop=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,min_delta=1e-5)

In [ ]:
reducelronplateau=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5)

In [ ]:
history=model.fit(x_t,y_t,validation_data=(x_v,y_v),epochs=60,batch_size=16,callbacks=[call,reducelronplateau])

In [ ]:
plt.figure()
accuracy=history.history['accuracy']
loss=history.history['loss']
val_acc=history.history['val_accuracy']
val_loss=history.history['val_loss']
epochs=[i for i in range(len(accuracy))]
plt.plot(epochs,loss,label='loss')
plt.plot(epochs,val_loss,label='val_loss')
plt.legend()

In [ ]:
test_f=[]
for i in test['Image']:
    a=i.split(" ")
    a= list(map(int, a)) 
    a=np.asarray(a).reshape(96,96)
    test_f.append(a)
    
test_=np.asarray(test_f)
test_final=np.expand_dims(test_,axis=3)
test_final.shape

In [ ]:
x=model.predict(test_final)

In [ ]:

count=2
xco=[]
yco=[]
for i in x[1]:
    if count%2==0:
        xco.append(i)
    else:
        yco.append(i)
    count+=1

In [ ]:
plt.imshow(np.squeeze(test_final[1]))
plt.scatter(xco,yco,color='red')